In [ ]:
install.packages("mvProbit")
library(mvProbit)
options( digits = 6 )
options(repr.matrix.max.cols = Inf)

In [ ]:
#read data created by "data processing"
d <- read.csv("D:/Processed Data/DE_ownership_input_30k.csv")
#d

#Throw out all columns not used
d_mvProbit <- subset(d, select = c(cbike, ebike, elevar, Raumtyp_12, Raumtyp_21, Raumtyp_22,  alter_gr_1, alter_gr_2, alter_gr_3, alter_gr_5, alter_gr_6, alter_gr_7, alter_gr_8, P_BIL_2, P_BIL_3, P_BIL_5, P_BIL_6, HP_SEX_2, taet_2, taet_3, taet_4, taet_5, oek_status_1, oek_status_2, oek_status_4, oek_status_5, hhgr_gr2_2, hhgr_gr2_3, hhgr_gr2_4))

#add column "constant" (always 1) to estimate ASC
nObs <- nrow(d_mvProbit)
const <- rep(1, nObs)
d_mvProbit <- cbind(d_mvProbit, const)

#create variables for interaction of age with occupation and level of education
d_mvProbit$P_BIL_2xadult <- d_mvProbit$P_BIL_2 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$P_BIL_2 <- NULL
d_mvProbit$P_BIL_3xadult <- d_mvProbit$P_BIL_3 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$P_BIL_3 <- NULL
d_mvProbit$P_BIL_5xadult <- d_mvProbit$P_BIL_5 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$P_BIL_5 <- NULL
d_mvProbit$P_BIL_6xadult <- d_mvProbit$P_BIL_6 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$P_BIL_6 <- NULL

d_mvProbit$taet_2xadult <- d_mvProbit$taet_2 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$taet_2 <- NULL
d_mvProbit$taet_3xadult <- d_mvProbit$taet_3 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$taet_3 <- NULL
d_mvProbit$taet_4xadult <- d_mvProbit$taet_4 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$taet_4 <- NULL
d_mvProbit$taet_5xadult <- d_mvProbit$taet_5 * (1- d_mvProbit$alter_gr_1)
d_mvProbit$taet_5 <- NULL

In [ ]:
#define starting values for betas; one beta more than parameters (for constant which is not called explicitly in mvProbit function)
#starting values from a past estimation
beta <- cbind(c(1.1,-3.3,-0.09,-0.01,0.1,0.27,-0.28,-0.15,-0.1,-0.19,-0.39,-0.98,-0.18,-0.096,0.12,-0.079,-0.18,0.05,-0.13,-0.18,-0.28,-0.12,-0.10,0.2,0.22,0.21,0.26,0.44),
             c(-1.9,0.67,0.11,0.21,0.12,-0.51,-0.61,-0.25,0.19,0.38,0.4,0.16,0.14,0.11,-0.04,0,-0.04,0.07,0.17,0.09,-0.03,-0.25,-0.19,0.03,0.16,0.27,0.11,0.13))

#define starting values for covariance of error terms (from past estimations)
sigma <- cbind(c(1,-0.238),
              c(-0.238,1))

In [ ]:
#mvProbit estimation
estResult <- mvProbit(cbind(cbike, ebike)~elevar+Raumtyp_12+Raumtyp_21+Raumtyp_22+alter_gr_1+alter_gr_2+alter_gr_3+alter_gr_5+alter_gr_6+alter_gr_7+alter_gr_8+P_BIL_2xadult+P_BIL_3xadult+P_BIL_5xadult+P_BIL_6xadult+HP_SEX_2+taet_2xadult+taet_3xadult+taet_4xadult+taet_5xadult+oek_status_1+oek_status_2+oek_status_4+oek_status_5+hhgr_gr2_2+hhgr_gr2_3+hhgr_gr2_4,
                     start = c(beta),
                      startSigma = sigma,
                      data = d_mvProbit, iterlim = 10, nGHK = 50,
                     method = "BHHH")

In [ ]:
summary(estResult)

In [ ]:
#Calculate null-log-likelihood

#data
d_mvProbit_0 <- d_mvProbit_30000

# model coefficients
beta_0 <- cbind(c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
               c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

# covariance matrix of error terms
sigma_0 <- cbind(c(1,0),
              c(0,1))

# log likelihood values
logLikVal <- mvProbitLogLik( cbind(cbike, ebike)~elevar+Raumtyp_12+Raumtyp_21+Raumtyp_22+alter_gr_1+alter_gr_2+alter_gr_3+alter_gr_5+alter_gr_6+alter_gr_7+alter_gr_8+P_BIL_2xadult+P_BIL_3xadult+P_BIL_5xadult+P_BIL_6xadult+HP_SEX_2+taet_2xadult+taet_3xadult+taet_4xadult+taet_5xadult+oek_status_1+oek_status_2+oek_status_4+oek_status_5+hhgr_gr2_2+hhgr_gr2_3+hhgr_gr2_4, 
   coef = c( beta_0 ), sigma = sigma_0, data = d_mvProbit_0 )

sum(logLikVal)

In [ ]:
#hessian(estResult)
#vcov( estResult, eigentol=1e-12)